In [1]:
import tensorflow as tf
from model_v2 import LanguageModel
import json
import numpy as np
import re
import pandas as pd
import unicodedata
import numpy as np
from utils import pad_sequences, clean_text

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [3]:
with open('102/word2idx.json', 'r') as inp:
    word2idx = json.load(inp)
with open('102/char2idx.json', 'r') as inp:
    char2idx = json.load(inp)
idx2word = {k: v for v, k in word2idx.items()}

In [4]:
with open('102/checkpoints/model_configs.json', 'r') as inp:
    params = json.load(inp)

In [5]:
model = LanguageModel(**params, is_training=False)

In [6]:
model.build_model()
saver = tf.train.Saver([x for x in tf.global_variables() if x not in tf.get_collection('LSTM_SAVED_STATE')])
sess.run(tf.global_variables_initializer())

In [7]:
saver.restore(sess, '102/checkpoints/test/model.cpkt-557539')

INFO:tensorflow:Restoring parameters from 102/checkpoints/test/model.cpkt-557539


In [8]:
def sample(a, temperature=0.7):
    a = a / temperature
    dist = np.exp(a) / np.sum(np.exp(a))
    choices = range(len(a))
    return np.random.choice(choices, p=dist)

In [9]:
def generate_text(initial_word, gen_len, reverse=False):
    init_word = clean_text(initial_word, add_bos=not reverse, add_eos=reverse).split()
    seq_lens = [len(init_word)]
    inputs = [[char2idx[y] for y in x] for x in init_word]
    maxlen = max(len(x) for x in inputs)
    arr = np.zeros(shape=(len(inputs), 1, maxlen), dtype=np.int32)
    for ir in range(len(inputs)):
        s = inputs[ir]
        arr[ir][0][:len(s)] = s
    inputs = arr
    if reverse:
        inputs = np.flip(inputs, axis=0)
        # Run the first time
        output = sess.run(model.bw_model['decoder'], feed_dict={model.bw_inputs: inputs, model.reset_state: True, model.seq_lens: seq_lens})
    else:
        output = sess.run(model.fw_model['decoder'], feed_dict={model.fw_inputs: inputs, model.reset_state: True, model.seq_lens: seq_lens})
    next_idx = sample(output[-1, 0, :])
    result = [
        idx2word[next_idx]
    ]
#     stop_token = word2idx['S']
    for i in range(1, gen_len):
        next_word = result[-1]
        inputs = [char2idx[y] for y in next_word]
        arr = np.zeros(shape=(1, 1, len(inputs)), dtype=np.int32)
        arr[0][0][:] = inputs
        inputs = arr
        if reverse:
            output = sess.run(model.bw_model['decoder'], feed_dict={model.bw_inputs: inputs, model.reset_state: False, model.seq_lens: [1]})
        else:
            output = sess.run(model.fw_model['decoder'], feed_dict={model.fw_inputs: inputs, model.reset_state: False, model.seq_lens: [1]})
        next_idx = sample(output[-1, 0, :])
        w = idx2word[next_idx]
#         print(w)
        result.append(w)
#         if next_idx == stop_token:
#             break
    return ' '.join((init_word[::-1] + result)[::-1]) if reverse else ' '.join(init_word + result)
        

In [10]:
print(generate_text('Bước vào vòng 2 Cincinnati Masters, Roger Federer đặt mục tiêu giành một chiến thắng thuyết phục. Bằng chứng là "Tàu tốc hành" đã tấn công tay vợt 29 tuổi Gojowczyk hết sức dữ dội.', 100, reverse=True))
print('-' * 100)
print(generate_text('Bước vào vòng 2 Cincinnati Masters, Roger Federer đặt mục tiêu giành một chiến thắng thuyết phục. Bằng chứng là "Tàu tốc hành" đã tấn công tay vợt 29 tuổi Gojowczyk hết sức dữ dội.', 100, reverse=False))

tả khoa học đầu tiên năm <number> . _nl_ </S> <S> Tứ kết Wimbledon , Federer vô địch Wimbledon <number> _nl_ Trước thềm giải thứ <number> , Roger Federer ( <number> tuổi ) đã vượt quá số <number> đội của FedEx để tiến tới danh hiệu lần thứ <number> của Nadal với tỷ số <number> - <number> ( <number> - <number> ) . Tuy nhiên , Federer vẫn chiến thắng sau <number> set với tỷ số <number> - <number> ( <number> - <number> ) và để mất break ở ván đấu đầu tiên ( <number> - <number> ) . _nl_ Bước vào vòng <number> Cincinnati Masters , Roger Federer đặt mục tiêu giành một chiến thắng thuyết phục . Bằng chứng là " Tàu tốc hành " đã tấn công tay vợt <number> tuổi Gojowczyk hết sức dữ dội . </S>
----------------------------------------------------------------------------------------------------
<S> Bước vào vòng <number> Cincinnati Masters , Roger Federer đặt mục tiêu giành một chiến thắng thuyết phục . Bằng chứng là " Tàu tốc hành " đã tấn công tay vợt <number> tuổi Gojowczyk hết sức dữ dội . Trướ

In [11]:
print(generate_text('Hôm nay tôi ăn cam', 30, reverse=True))
print('-' * 100)
print(generate_text('Hôm nay tôi ăn cam', 30, reverse=False))

nhận thấy sự trong sáng của mọi người trong xã hội là rất quan trọng . Tôi rất quan tâm đến kiến thức về văn hóa của người trẻ . Hôm nay tôi ăn cam </S>
----------------------------------------------------------------------------------------------------
<S> Hôm nay tôi ăn cam lòng ! Nhưng tôi vẫn có được một tấm ảnh đẹp . Tôi chụp ảnh cho cả nhà , từng người từng bu từng món quà mà tôi rất trân


In [12]:
print(generate_text('Bưởi là', 30, reverse=False))

<S> Bưởi là một trong những lý do giúp đề phòng hiệu quả . Đúng là đã có một bông hoa mới , một loại hoa mà được các bà hoàng khen cho


In [13]:
print(generate_text('Chuối là', 30, reverse=False))

<S> Chuối là một trong những loại cây phổ biến nhất , được chế biến nhất trong các loại cây , đậu , quả . Loại chuối này còn có nhiều loại chất


In [14]:
print(generate_text('Roger Federer', 100, reverse=False))

<S> Roger Federer , người làng <UNK> , vùng <UNK> <UNK> , thành phố <UNK> , bang Maine , Mỹ , là dân số của làng <UNK> . <UNK> là một khu làng không có điện ảnh , và lại là nơi sinh hoạt của <number> ngàn sinh viên , người dân thành phố New <UNK> . <UNK> <UNK> <UNK> <UNK> <UNK> ( <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> ) <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>


In [15]:
generate_text('tay vợt người anh andy murray', 100)

'<S> tay vợt người anh andy murray <UNK> , từng tập huấn các kỹ thuật hình ảnh và các cú đại dương ở các sân vận động . Sau <number> tập đầu , chương trình sẽ tiến hành các trận đấu với <number> VĐV dưới tên tuổi Kalashnikov . Tại vòng loại này , ngôi sao <number> tuổi người Nhật Bản đã giúp giành vé dự vòng chung kết sau World Cup <number> . Ở một giải đấu đặc biệt , <UNK> đặt áo vào vòng hai của đội tuyển Nhật Bản và thắng <number> - <number> . Đây là lần đầu tiên đội tuyển VN được'

In [16]:
generate_text('công nghệ blockchain', 100, False)

'<S> công nghệ blockchain _nl_ Lịch sử . _nl_ <UNK> là một bộ não của một số động vật <UNK> . _nl_ </S> <S> <UNK> _nl_ <UNK> là một làng thuộc huyện Chrudim , vùng Pardubický , Cộng hòa Séc . _nl_ </S> <S> <UNK> _nl_ <UNK> là một đô thị thuộc huyện Dithmarschen , trong bang Schleswig - Holstein , nước Đức . Đô thị <UNK> có diện tích <number> , <number> km² , dân số thời điểm cuối năm <number> là <number> người . _nl_ </S> <S> <UNK> , Syunik _nl_ <UNK> là một xã thuộc huyện Domaniç , tỉnh Malatya ,'

In [17]:
generate_text('Sony', 100, False)

'<S> Sony và những chiếc máy bay của hãng này đã bị xáo trộn mạnh mẽ với các máy bay chiến đấu Anh - Nga của hãng Air . Hãng hàng không Airbus đã gọi điện về câu trả lời cho những người nổi tiếng như là một người đàn ông tự do . Họ bắt đầu được giữ cho đội bay của Hãng hàng không <UNK> . Hàng không nói họ cũng không biết mặt , nhưng sẽ có đống máy bay chạy ngược lại , sau khi kích hoạt lại một chiếc máy bay . Trong khi , Hãng hàng'

In [18]:
print(generate_text('Sony cho ra mắt điện thoại Xperia Z3 tại CES năm nay', 100))

<S> Sony cho ra mắt điện thoại Xperia Z3 tại CES năm nay . _nl_ Sau đó , <UNK> sẽ ra mắt mẫu <UNK> Avant ( <UNK> ) , sau đó và sẽ được phát hành dưới dạng <UNK> . Tuy nhiên , nguồn tin của nhà sản xuất <UNK> cho rằng , chính phủ máy tính đưa ra đã ra " <UNK> " nhờ công nghệ <UNK> <UNK> mang tên <UNK> <UNK> . Ngoài ra , <UNK> còn có nhiều ưu điểm khác biệt : mở lên và mở các thiết bị ( <UNK> ) , <UNK> ( <UNK> ) , <UNK> ( <UNK> ) , <UNK> ( <UNK> <UNK> <UNK> <UNK>


In [19]:
generate_text('bệnh viện bạch mai cho biết', 100)

'<S> bệnh viện bạch mai cho biết hiện có khoảng <number> . <number> ca chất thuốc được sử dụng cho việc chữa bệnh . Theo một nghiên cứu mới đây của Đại học Y khoa tại Đại học <UNK> tại thành phố <UNK> , <number> % bệnh nha ( <UNK> - <UNK> ) vẫn diễn ra với bệnh nhân . Theo báo cáo của Hiệp hội Y học thế giới , tại đây có khoảng <number> . <number> bệnh nhân bị đái , <number> . <number> người đang trong giai đoạn điều trị . </S> <S> <UNK> <UNK> _nl_ <UNK> <UNK> là một loài bướm đêm trong'

In [20]:
generate_text('Đại học Bách Khoa', 100, True)

'<UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> qua <UNK> . Tuy nhiên , ở đây , những cảnh quan đền cổ và di tích khác vẫn chưa có ghi nhận khác về giá trị di sản kiến trúc cổ của thành phố này là <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> , do trường Đại học Bách Khoa </S>'

In [21]:
generate_text('các doanh nghiệp', 200)

'<S> các doanh nghiệp trong lĩnh vực nông nghiệp , nông lâm phải chịu nhiều tiêu chí như đảm bảo an toàn , tạo ra thông tin và hình thức cho thị trường . </S> <S> <UNK> <UNK> _nl_ <UNK> <UNK> là một loài thực vật có hoa trong họ Cúc . Loài này được ( Lour ) Grau mô tả khoa học đầu tiên năm <number> . _nl_ </S> <S> <UNK> , İskilip _nl_ <UNK> là một xã thuộc huyện İskilip , tỉnh Çorum , Thổ Nhĩ Kỳ . Dân số thời điểm năm <number> là <number> người . _nl_ </S> <S> <UNK> _nl_ <UNK> là một đô thị ở huyện Schmalkalden - Wittlich , trong bang Schleswig - Holstein , nước Đức . Đô thị <UNK> có diện tích <number> , <number> km² , dân số thời điểm ngày <number> tháng <number> năm <number> là <number> người . _nl_ </S> <S> <UNK> <UNK> _nl_ <UNK> <UNK> là một loài bướm đêm trong họ Noctuidae . _nl_ </S> <S> <UNK> , Tumkur _nl_ <UNK> là một làng thuộc tehsil Tumkur , huyện Tumkur , bang Karnataka , Ấn . _nl_ </S> <S> <UNK> _nl_ <UNK> là một thị xã và đô thị của'

In [22]:
generate_text('nguyễn phú trọng', 50)

'<S> nguyễn phú trọng , không thể bỏ qua . Hầu hết chúng tôi đều đi dạy , học tiếng Anh , học sinh , thi thức ; các buổi học , buổi chiều học và thi ! . Sáng <number> - <number> , chúng tôi dự lễ hội của các học sinh và'

In [23]:
generate_text('một cô gái được các chàng trai yêu thích', 200)

'<S> một cô gái được các chàng trai yêu thích trong việc chụp ảnh cùng bình luận . Các nàng , tôi sẽ tăng sức mạnh của mình cho người đẹp và các em truyền thống của mình . </S> <S> <UNK> <UNK> _nl_ <UNK> <UNK> là một loài bướm đêm trong họ Geometridae . _nl_ </S> <S> Tại buổi làm việc chiều <number> - <number> , đại diện Sở Tài nguyên - môi trường TP Đà Nẵng cho biết ngày <number> - <number> , báo Tuổi Trẻ đã có quyết định xử phạt đối với Công ty TNHH <UNK> Việt Nam về việc bồi thường thiệt hại cho Công ty TNHH xây dựng và thương mại <UNK> . Theo đó , Công ty TNHH Dịch vụ và thương mại <UNK> Việt Nam được miễn tiền bồi thường cho những người bị nạn bằng cách sử dụng tiền mặt ( <number> triệu đồng ) để phân chia di sản , thu lợi tức cho dân chủ . </S> <S> <UNK> , Mulbagal _nl_ <UNK> là một làng thuộc tehsil Chintamani , huyện Kolar , bang Karnataka , Ấn Độ . _nl_ </S> <S> <UNK> <UNK> _nl_ <UNK> <UNK> là một loài bướm đêm trong họ Crambidae . _nl_ </S> <S>'

In [24]:
generate_text('mạng xã hội facebook', 50)

'<S> mạng xã hội facebook . com . vn . vn . Những lời phụ thân trên mạng xã hội Nguồn tin từ Quỹ <UNK> Research cho biết trong năm đó , không ít người đã đọc những bài bình luận về cuộc sống của người dân . Họ thu chi không đủ để người'

In [25]:
generate_text('samsung galaxy', 50)

'<S> samsung galaxy <UNK> <number> . <number> ( loại máy bay <number> ) có sức mạnh <number> , <number> năm và có thể bay không dưới <number> cm ( <number> . <number> m ) . Tổng cộng có <number> chiếc được sản xuất vào tháng <number> - <number> . <number> chiếc còn lại'

In [26]:
generate_text('Nhận định ban đầu về vụ cả làng nhiễm HIV', 100)

'<S> Nhận định ban đầu về vụ cả làng nhiễm HIV / AIDS <number> : Hai được đánh giá là một trong những vấn đề về tâm lý không thể lột tả hết . Để có được một cái nhìn đẹp , biết kết quả của cuộc sống , nhiều người nói rằng , không có gì là dễ dàng hơn , nhưng có thực lại là một thứ việc bán đi . Có lẽ bạn sẽ nghĩ gì . Bạn có thể nghĩ mình là một người , nhưng cũng có nhiều người để cho người khác . <number> / <number> . Người nghiện . Người ta có tình yêu'

In [27]:
generate_text('Điện Biên: Khởi tố vụ án nổ súng làm chết 3 người tại nhà riêng', 400)

'<S> Điện Biên : Khởi tố vụ án nổ súng làm chết <number> người tại nhà riêng ở xã Vĩnh Hòa huyện Vĩnh Tường , tỉnh Vĩnh Long . Vụ án mạng xảy ra ở xã Vĩnh Hòa , huyện Cư Chải . Cơ quan điều tra đã làm rõ hành vi của một đối tượng . Theo đó , <number> đối tượng tên Nguyễn Trọng Hoàng , Đỗ Duy Linh , Nguyễn Thị Toàn , Nguyễn Văn Tuấn , Nguyễn Văn Chiến , Nguyễn Quốc Tuấn , Nguyễn Ngọc Tuấn , Phạm Văn Nguyên , Nguyễn Quốc Tuấn , Nguyễn Đức Việt ( tức Huỳnh Minh Phước ) , Nguyễn Văn Tuân ( <number> tuổi , trú xã Đại Dương , huyện Vĩnh Cửu ) , Nguyễn Thái Sáng , Nguyễn Văn Trung ( <number> tuổi , xã Vĩnh Hiệp ) , Nguyễn Văn Toản ( <number> tuổi , trú nhà lầu <number> ) và Nguyễn Văn Tư ( <number> tuổi ) , Nguyễn Văn Bình ( <number> tuổi , cùng ngụ tại xã Long Phú ) , Nguyễn Đức Trung ( <number> tuổi , ngụ tại xã Vĩnh Bình , huyện Từ Liêm ) . Cả hai nhóm đối tượng đang phân vân đánh bạc và đánh <number> nhát vào nhóm người này . Sau đó , nhóm này bỏ chạy về nhà , làm nhiều người chết . Được 

In [28]:
generate_text('Giá vàng lao dốc mạnh, bất ổn Thổ Nhĩ Kỳ do FED tăng lãi suất?', 400)

'<S> Giá vàng lao dốc mạnh , bất ổn Thổ Nhĩ Kỳ do FED tăng lãi suất ? _nl_ Giá vàng hôm nay đã giảm <number> , <number> % xuống còn <number> , <number> USD / ounce vào hồi đầu tuần . Giá vàng thế giới giảm nhẹ , chỉ số <number> , <number> % , xuống mức <number> , <number> USD / ounce . Giá vàng đã giảm <number> , <number> % xuống <number> , <number> USD / ounce . Giá vàng giao ngay giảm <number> , <number> % xuống <number> , <number> USD / ounce , mức thấp nhất kể từ <number> , <number> USD / ounce , tính tới <number> / <number> . Tính chung cả tuần trước , giá vàng SJC tuần qua không còn ở mức trần . Giá vàng thế giới cũng giảm <number> , <number> USD xuống <number> , <number> USD / ounce trong khi chỉ số các phiên giao dịch hôm qua tăng <number> , <number> USD / ounce so với chốt phiên hôm qua . Vàng tăng trở lại là do giá tăng mạnh nhất trong tháng <number> / <number> . Theo niềm tin của giới đầu tư , giá vàng thế giới đã tăng <number> , <number> % trong vòng <number> năm qua . Đồng

In [29]:
generate_text('Một app khá hay cho anh em đang làm nhà sửa nhà hoặc chuẩn bị xây nhà sửa nhà, anh em tham khảo nhé!', 100)

'<S> Một app khá hay cho anh em đang làm nhà sửa nhà hoặc chuẩn bị xây nhà sửa nhà , anh em tham khảo nhé ! Tổng hợp </S> <S> <UNK> , Ereğli _nl_ <UNK> là một xã thuộc huyện Güroymak , tỉnh Mardin , Thổ Nhĩ Kỳ . Dân số thời điểm năm <number> là <number> người . _nl_ </S> <S> <UNK> <UNK> _nl_ <UNK> <UNK> là một loài ốc trong họ <UNK> , là động vật thân mềm chân bụng sống ở biển trong họ Turridae . _nl_ </S> <S> <UNK> _nl_ <UNK> là một xã ở tỉnh Haute - Saône trong vùng Bourgogne - Franche - Comté phía đông nước Pháp . _nl_ </S> <S> <UNK> , Bodrum _nl_ <UNK> là một xã'

In [30]:
generate_text('Uống một chút rượu mỗi ngày có thể giúp ngăn chặn chứng mất trí nhớ', 300)

'<S> Uống một chút rượu mỗi ngày có thể giúp ngăn chặn chứng mất trí nhớ của người khác khi ngày càng tăng . Thông thường , việc uống rượu bia vào thứ bảy , nhưng không phải là không có thời gian . Nhiều người thường nghĩ ra rượu là uống đủ và sau khi uống rượu , uống rượu phải có thêm nước , uống bia , uống nước hoa . . . Mỗi ngày cứ về một loại rượu , pha <UNK> nước . Vì vậy , uống rượu không cần tôi uống rượu , uống . Uống rượu phải uống nước gừng , mặt , có bụng , phải uống rượu nhiều hơn . Rượu bia là điều quan trọng . Rượu cần làm từ rượu , rượu , uống nước . Mỗi bữa uống của người khác được coi là vô ích . Nếu không nhậu thì không tốt , không được ăn quá , uống rượu không no . Uống rượu quá là phải lớn . Bởi cuộc sống không bao giờ no , no , không có máu . Ăn uống rượu bia ! Uống rượu , uống rượu thì làm ra . Uống bia rượu nhiều , uống nhiều lắm . Uống rượu vì uống rượu quá gì , uống bia cao quá , uống rượu bia thì uống bia . Uống rượu thì khó , vì uống rượu say cũng khó . Ăn uố